<a href="https://colab.research.google.com/github/pszachew/Music-Genre-Classification/blob/master/training_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
X = pd.read_csv('/content/gdrive/MyDrive/csv_data/embedded.csv',header=None)

In [4]:
y = pd.read_csv('/content/gdrive/MyDrive/csv_data/labels.csv', header=None)

In [17]:
scaler = StandardScaler()
scaler.fit(X)
X_sc =pd.DataFrame(scaler.transform(X))

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X_sc,y,test_size=0.3, random_state=10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,30680,30681,30682,30683,30684,30685,30686,30687,30688,30689,30690,30691,30692,30693,30694,30695,30696,30697,30698,30699,30700,30701,30702,30703,30704,30705,30706,30707,30708,30709,30710,30711,30712,30713,30714,30715,30716,30717,30718,30719
0,4.383885,-0.166742,1.485253,-0.571546,0.114628,-0.551052,-0.572712,0.937984,-0.189045,2.939065,-1.245144,2.872746,0.025463,-0.261930,-0.033067,1.587075,-0.733359,0.945534,-1.898769,-0.310077,-0.589116,-0.316174,-0.740309,2.571831,1.796360,-0.191272,0.430013,-0.679932,0.649041,-0.011235,1.026029,0.384278,-0.811171,-0.529569,0.656915,0.718845,-0.301080,-0.829768,-0.891711,0.298833,...,-1.114785,-0.009918,0.113695,0.392770,-0.095643,0.507262,0.657085,-0.710002,-0.559200,-0.500389,0.136353,1.640595,-0.885300,1.051221,-0.279141,-1.086439,0.314005,0.137050,0.256497,-0.271425,0.456396,0.016003,-0.555036,-1.720807,-0.320984,1.652521,0.072079,-1.190968,-0.645239,-0.864008,-0.268315,0.122180,-0.958708,-0.324965,-0.539149,-0.817587,-0.403436,-0.589506,-0.907549,-0.146531
1,-0.138930,-1.134390,0.040051,-0.729560,-1.133774,-0.008232,-0.084163,0.828403,-0.189045,1.804787,0.784094,-0.800590,-0.492581,-0.215538,0.042966,-0.968081,-0.580512,1.945759,-0.978987,0.766338,0.373474,0.274545,-0.808763,2.608353,0.637039,0.162240,0.736944,2.198596,-0.830520,0.423442,-0.166771,1.501655,0.797500,1.495283,-0.707097,-0.769880,-0.100208,-0.829768,-0.285866,0.573162,...,0.778508,-0.215015,0.067824,0.869694,-1.440242,-0.866191,1.373365,-0.290304,0.003720,0.894602,-0.414708,0.007899,-0.367050,-0.134195,-0.933886,-0.454633,0.809731,-0.013660,-0.064331,0.774829,0.004588,-0.050240,0.739694,1.010528,-0.197082,1.217739,1.180362,1.018017,0.648563,-0.818498,0.194994,-0.433980,1.433355,-0.000580,-0.418577,0.321285,0.005975,-0.311840,-1.020490,-0.001394
2,-0.648994,-1.587555,0.789500,0.239021,-0.162165,-0.122665,3.290068,-0.163981,-0.189045,-0.422789,1.110349,0.632571,0.401887,0.435341,0.134473,-1.188181,0.558228,0.563337,-0.000614,0.493841,0.360617,-0.002541,-0.736860,0.470317,0.591919,0.484000,1.782646,1.304730,0.798112,1.958114,1.939970,1.102766,0.085464,0.227854,-0.000056,0.964424,-1.241659,1.715525,-1.191595,0.211754,...,-0.479047,0.443118,-0.456292,0.770761,-1.270853,-0.079474,-0.543178,-0.554987,-1.211819,-0.248648,-0.272600,0.144650,-0.351904,-1.028814,-0.655957,0.259125,-0.404364,-0.280666,-0.552925,1.070614,-0.708768,-0.664495,0.501352,-0.246586,-0.591352,1.613641,-0.346904,-0.511161,-0.643534,-1.699378,0.559520,0.372802,1.053894,0.180347,-1.189915,-0.232068,-0.216206,0.840709,1.107061,-0.437145
3,2.333005,0.502068,2.299781,-0.727548,-1.639913,-0.704098,1.478860,0.355287,-0.189045,-0.422789,-1.778942,-1.467001,-0.511789,-1.624437,-0.135239,2.467595,-0.733359,2.370035,0.798741,-0.310077,-0.492456,-0.344479,3.968705,0.238681,0.610373,-0.317285,-0.365263,-0.328090,-1.596599,0.136854,0.549394,0.422909,-0.143828,-0.963458,-0.536288,0.438638,-1.001089,2.224639,1.437499,-0.860312,...,1.729810,-0.383018,0.745074,-0.052388,0.570329,-0.765872,0.502122,0.198112,-0.026272,-1.246490,-0.543540,0.589992,0.379787,-0.025453,0.955230,0.712015,-0.367515,-0.537002,-0.036721,1.649872,0.666862,-1.250410,-0.930453,-1.259191,0.378049,0.027001,-0.213115,1.418526,-0.909041,-0.919578,-0.334989,0.328667,1.744387,-0.008229,-0.641904,1.148128,-0.266780,-0.383235,0.298152,0.118918
4,-0.336561,-0.415097,0.305132,-0.341592,0.486421,-0.044456,0.630534,0.858402,-0.189045,-0.422789,0.371410,1.207356,-0.709775,-0.807979,-1.624174,0.451441,0.747375,0.828938,0.192945,-0.310077,1.746914,-0.344262,-0.099298,0.238727,2.093200,-1.154810,-0.616554,2.150519,-1.048941,2.080376,-0.518240,-0.586726,0.385799,-1.065644,-0.825524,-0.541598,-0.168869,-0.829768,-1.191595,-0.860312,...,1.085095,-0.689365,-0.324356,-1.215419,1.326814,-0.324360,-0.194912,-0.485639,-0.371205,0.605217,1.336355,0.253417,-0.823760,1.170021,1.676653,-0.902502,-0.292353,0.980044,-0.615

## **Logistic Regression**

In [23]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_reg_mod = LogisticRegression(max_iter=1000)

In [ ]:
log_reg_mod.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
joblib.dump(log_reg_mod, 'log_reg.sav')

['log_reg.sav']

In [ ]:
y_pre_lg = log_reg_mod.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pre_lg))

              precision    recall  f1-score   support

       blues       0.85      0.94      0.89        35
   classical       0.92      1.00      0.96        33
     country       0.74      0.74      0.74        27
       disco       0.86      0.74      0.79        34
      hiphop       0.74      0.96      0.84        24
        jazz       1.00      0.91      0.96        35
       metal       0.97      1.00      0.98        31
         pop       0.79      0.88      0.84        26
      reggae       0.92      0.77      0.84        30
        rock       0.70      0.56      0.62        25

    accuracy                           0.86       300
   macro avg       0.85      0.85      0.85       300
weighted avg       0.86      0.86      0.85       300



In [ ]:
print(confusion_matrix(y_test, y_pre_lg))

[[33  1  0  0  0  0  0  0  0  1]
 [ 0 33  0  0  0  0  0  0  0  0]
 [ 3  0 20  2  0  0  0  1  0  1]
 [ 0  0  2 25  2  0  0  2  0  3]
 [ 0  0  0  0 23  0  0  0  1  0]
 [ 1  2  0  0  0 32  0  0  0  0]
 [ 0  0  0  0  0  0 31  0  0  0]
 [ 0  0  0  0  2  0  0 23  0  1]
 [ 0  0  2  1  2  0  0  2 23  0]
 [ 2  0  3  1  2  0  1  1  1 14]]


In [29]:
kf3 = KFold(n_splits=3, shuffle=True)
mod_lg = LogisticRegression(max_iter=10000)

In [30]:
i=1
for x_train_idx, x_test_idx in kf3.split(X_sc):
  X_train = X_sc.iloc[x_train_idx]
  X_test = X_sc.iloc[x_test_idx]
  y_train = y.iloc[x_train_idx]
  y_test = y.iloc[x_test_idx]
  mod_lg.fit(X_train, y_train)
  print(f"Accuracy for the fold no. {i} on the test set: {accuracy_score(y_test, mod_lg.predict(X_test))}")
  i=i+1




/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy for the fold no. 1 on the test set: 0.8618618618618619


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy for the fold no. 2 on the test set: 0.8468468468468469


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy for the fold no. 3 on the test set: 0.8618618618618619


## **KNN**

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [33]:
neigh = KNeighborsClassifier(n_neighbors = 4)
neigh.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                     weights='uniform')

In [37]:
acc_ls = []

In [40]:
for n in range (3,12):
  neigh = KNeighborsClassifier(n_neighbors = n)
  neigh.fit(X_train, y_train)
  acc_ls.append(accuracy_score(y_test, neigh.predict(X_test)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A colum

In [41]:
acc_ls

[0.8033333333333333,
 0.81,
 0.79,
 0.8,
 0.7933333333333333,
 0.8033333333333333,
 0.81,
 0.79,
 0.8,
 0.7933333333333333,
 0.8,
 0.7833333333333333,
 0.78,
 0.7933333333333333]